In [1]:
import numpy as np
import pandas as pd
import os
import glob
import re
import math


In [2]:
os.chdir('/Users/lorenzoflores/Desktop/Nerve/Hospitals/')

In [3]:
path = '/Users/lorenzoflores/Desktop/Nerve/Hospitals/'
extension = 'csv'
os.chdir(path+'DOH_List')
titles = glob.glob('*.{}'.format(extension))


## Goals
* Overall Plot: Provinces as outlines, regions as colors, points as hospitals
* Hospital Deserts: Municipalities as outlines, population density as colors, points as hospitals
* Municipalities by Distance: Munipalities as outlines, distance to nearest hospital as colors, points as hospitals

## Datasets

### `data` 
From DOH: https://nhfr.doh.gov.ph/rfacilities2list.php
   * `Health Facility Code Short`: ID
   * `Facility Name`: Name
   * `Old Health Facility Names`: Old Name
   * `Old Health Facility Name 2`: Old Name 2
   * `Health Facility Type`: 'Ambulatory Surgical Clinic', 'Animal Bite Treatment Center', 'Barangay Health Station', 'Birthing Home', 'City Health Office', 'DepEd Clinic', 'Dialysis Clinic', 'Drug Abuse Treatment and Rehabilitation Centers', 'Drug Testing Laboratory', 'General Clinic Laboratory', 'Hospital', 'Infirmary', 'Municipal Health Office', 'Provincial Health Office', 'Psychiatric Care Facility', 'Rural Health Unit', 'Social hygiene Clinic'
   * `Ownership Major Classification`: 'Private', 'Government'
   * `Ownership Sub-Classification for Government facilities`: nan, 'University Hospital', 'Department of Justice', 'Office of the President', ' ', 'Local Government Unit', 'Philippine National Police', 'Department Of Health', 'National Government', 'Department of National Defense'
   * `Ownership Sub-Classification for private facilities`: nan, 'Foundation', 'Corporation', 'Religious', 'Cooperative', ' ', 'Single Proprietorship', 'Non-government Organization'
   * `Street Name and #          `: Address
   * `Building name and #`: Building Address
   * `Region Name`: Region
   * `Region PSGC`: Region Code (10,000,000 to 170,000,000)
   * `Province Name`: Province
   * `Province PSGC`: Province Code
   * `City/Municipality Name`: City
   * `City/Municipality PSGC`: City Code
   * `Barangay Name`: Brgy
   * `Barangay PSGC`: Brgy Code
   * `Zip Code`: Zip Code
   * `Official Website`: Website
   * `Hospital Licensing Status`: nan, 'Not Applicable', 'With License', 'Without License'
   * `Service Capability`: nan, 'Level 3', 'Level 1', 'Level 2', 'Custodial Psychiatric Care'
   * `Bed Capacity`: Integer
    
### `psgc` 
From Thinking Machines: https://github.com/thinkingmachines/psgc
   * `interlevel`: Bgy, City, Mun, SubMun, etc.
   * `code`: Philippine Standard Geographic Code (PSGC) number
   * `location`: Name of Place
    
### `mun` 
From Wikipedia: https://en.wikipedia.org/wiki/List_of_cities_and_municipalities_in_the_Philippines#cite_note-PSA15%E2%80%93d-1
   * `City or municipality`: Name
   * `Population`: Number of People (2015)
   * `Area`: Land Area (km^2)
   * `PD`: Population Density (pop_2015/km^2)
   * `Barangay`: Number of Barangays
   * `Class`: Municipality, Highly Urbanized City, Component City, Independent Component City

## Hospitals

In [3]:
data = pd.read_csv(titles[0])
for item in titles[1:]:
    data = data.append(pd.read_csv(item), sort=False)

In [4]:
data = data[['Health Facility Code Short',
'Facility Name',
'Old Health Facility Names', 
'Old Health Facility Name 2',
'Health Facility Type',
'Ownership Major Classification',
'Ownership Sub-Classification for Government facilities',
'Ownership Sub-Classification for private facilities',
'Street Name and #           ',
'Building name and #',
'Region Name',
'Region PSGC',
'Province Name',
'Province PSGC',
'City/Municipality Name',
'City/Municipality PSGC',
'Barangay Name',
'Barangay PSGC',
'Zip Code',
'Official Website',
'Hospital Licensing Status',
'Service Capability',
'Bed Capacity']]

In [5]:
data = data.reset_index()

#### Filtering for Hospitals and Other Facilities

* Among the entries in `data`, only 1302 had service capability ratings (Level 1, 2, 3, or Psychiatric Care)
* 1299/1302 were hospitals, among these 343 had level 2 ratings while 118 had level 3 ratings
* Since only level 2 or 3 facilities can accept Covid-19 patients, it makes sense to focus on hospitals
* Other facilities (i.e. 'Hospital','Infirmary','General Clinic Laboratory', 'City Health Office', 'Municipal Health Office', 'Provincial Health Office') were included for comparison, to see how much more the reach could be if these facilities had the right equipment. However, rural health facilities and barangay health units (which composed most of the entries) were excluded for time and scope purposes

In [6]:
data['Service Capability'] = data['Service Capability'].fillna(0)

In [7]:
data.loc[:,['Service Capability','Health Facility Code Short']].groupby('Service Capability').count()

,Health Facility Code Short
Service Capability,
0,28677
Custodial Psychiatric Care,3
Level 1,838
Level 2,343
Level 3,118


In [8]:
data.loc[:,['Health Facility Type','Health Facility Code Short']].groupby('Health Facility Type').count()

,Health Facility Code Short
Health Facility Type,
Ambulatory Surgical Clinic,2
Animal Bite Treatment Center,4
Barangay Health Station,23071
Birthing Home,1903
City Health Office,11
DepEd Clinic,6
Dialysis Clinic,4
Drug Abuse Treatment and Rehabilitation Centers,59
Drug Testing Laboratory,24


In [9]:
data['target'] = [1 if item in ['Hospital','Infirmary','General Clinic Laboratory', 
               'City Health Office', 'Municipal Health Office', 
               'Provincial Health Office'] else 0 for item in data['Health Facility Type']]
subData = data[data['target']==1]
subData = subData.reset_index()

#### Creating Geotags for Facilities

In [19]:
# adds = []
# for i in range(subData.shape[0]):
#     add = subData['Facility Name'][i]
    
#     try:
#         city = subData['City/Municipality Name'][i]
#         city = city.replace('Capital','')
#         res = add+', '+city+', '+'Philippines'
#         adds.append(res)
#         continue
#     except:
#         pass
    
#     try: 
#         brgy = subData['Barangay Name'][i]
#         res = add+', '+brgy+', '+'Philippines'
#         adds.append(res)
#         continue
#     except:
#         res = add+', '+'Philippines'
#         adds.append(res)
#         continue

In [230]:
import json
import requests

def geoTag(string):
    try:
        response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+
                                string+'&key='+"")
        resp_json_payload = response.json()
        lat = resp_json_payload['results'][0]['geometry']['location']['lat']
        lon = resp_json_payload['results'][0]['geometry']['location']['lng']
        return [lat,lon]
    except:
        return 0


In [11]:
# subData['lat'] = np.repeat(None, subData.shape[0])
# subData['lon'] = np.repeat(None, subData.shape[0])

In [25]:
# brgyCoords = {}
# cityCoords = {}
# hospCoords = {}

In [26]:
# errNo = []
# for i in range(subData.shape[0]):
#     if i%100 == 0:
#         print("Now on: "+str((i/subData.shape[0])*100)+"%")
    
#     if adds[i] in hospCoords:
#         subData.loc[i,'lat'] = hospCoords[adds[i]][0]
#         subData.loc[i,'lon'] = hospCoords[adds[i]][1]
#         continue
#     else:
#         temp = geoTag(adds[i])
#         if temp != 0:
#             subData.loc[i,'lat'] = temp[0]
#             subData.loc[i,'lon'] = temp[1]
#             hospCoords[adds[i]] = temp
#             continue
        
#     # The full name was unable to be tagged, try city
#     if str(subData['Barangay Name'][i]) != 'nan':
#         if subData['Barangay Name'][i] in brgyCoords.keys():
#             subData.loc[i,'lat'] = brgyCoords[subData['Barangay Name'][i]][0]
#             subData.loc[i,'lon'] = brgyCoords[subData['Barangay Name'][i]][1]
#         else:
#             brgy = 'Barangay '+subData['Barangay Name'][i]+', '+subData['Province Name'][i]+', '+'Philippines'
#             temp = geoTag(brgy)
#             if temp != 0:
#                 subData.loc[i,'lat'] = temp[0]
#                 subData.loc[i,'lon'] = temp[1]
#                 brgyCoords[subData['Barangay Name'][i]] = temp
#                 continue
    
#     if subData['City/Municipality Name'][i] in cityCoords.keys():
#         subData.loc[i,'lat'] = cityCoords[subData['City/Municipality Name'][i]][0]
#         subData.loc[i,'lon'] = cityCoords[subData['City/Municipality Name'][i]][1]
#     else:
#         city = subData['City/Municipality Name'][i] +', '+'Philippines'
#         temp = geoTag(city)
#         if temp != 0:
#             subData.loc[i,'lat'] = temp[0]
#             subData.loc[i,'lon'] = temp[1]
#             cityCoords[subData['City/Municipality Name'][i]] = temp
#         else:
#             print('Error on '+str(i)+': '+subData['Facility Name'][i])
#             errNo.append(i)

# brgyCoords = pd.DataFrame(brgyCoords)
# cityCoords = pd.DataFrame(cityCoords)
# hospCoords = pd.DataFrame(hospCoords)


Now on: 0.0%
Now on: 4.391743522178305%
Now on: 8.78348704435661%
Now on: 13.175230566534916%
Now on: 17.56697408871322%
Now on: 21.958717610891522%
Now on: 26.35046113306983%
Now on: 30.742204655248134%
Now on: 35.13394817742644%
Now on: 39.52569169960474%
Now on: 43.917435221783045%
Now on: 48.309178743961354%
Now on: 52.70092226613966%
Now on: 57.092665788317966%
Now on: 61.48440931049627%
Now on: 65.87615283267458%
Now on: 70.26789635485288%
Now on: 74.65963987703118%
Now on: 79.05138339920948%
Now on: 83.44312692138779%
Now on: 87.83487044356609%
Now on: 92.22661396574439%
Now on: 96.61835748792271%


In [27]:
# brgyCoords.to_csv('brgyCoords.csv')
# cityCoords.to_csv('cityCoords.csv')
# hospCoords.to_csv('hospCoords.csv')

In [ ]:
# subData.to_csv('subData.csv')
# hospitals = subData[subData['Health Facility Type']=='Hospital']
# hospitals.to_csv("Hospitals.csv")

## Municipalities
* Merge municipality name, PSGC code, and population density
* Merge to shapefile for plotting of colors

Standardizing
* All names of municipalities are capitalized
* All names of provinces are in title case
* Class uses only either `City` or `Municipality`
* Use `location` for city/municipality name, `Province` for province name, and `interlevel` for class

In [7]:
mun = pd.read_csv("/Users/lorenzoflores/Desktop/Nerve/Hospitals/Municipalities.csv")
psgc = pd.read_csv("/Users/lorenzoflores/Desktop/Nerve/Hospitals/clean-psgc.csv")

In [8]:
mun[mun['City or municipality']=='Quezon City']

,City or municipality,Population,Area,PD,Barangay,Class,Province,Unnamed: 7,Unnamed: 8,Unnamed: 9
1034,Quezon City,2936116.0,165.33,17759.0,142.0,HUC,"NCR, 2nd district",NaN,NaN,NaN


### `psgc` data
#### Adding provinces to `psgc` data

In [9]:
provLookup = psgc[psgc['interlevel']=='Prov']
provLookup = provLookup[['code','location']]

In [10]:
provLookup = provLookup.rename(columns={'location':'Province', 'code': 'provCode'})

In [11]:
psgc.loc[psgc['interlevel']=='SubMun','code'] = 133900000
psgc.loc[psgc['interlevel']=='SubMun','interlevel'] = 'City'


In [12]:
psgc = psgc[psgc['interlevel'].isin(['City','Mun'])]

In [13]:
psgc['provCode'] = [100000*math.floor(item) for item in psgc['code']/100000]

In [14]:
provLookup = provLookup.append(pd.DataFrame({'provCode':[133900000,137400000,137500000,137600000],
                                             'Province':['MANILA','MANILA','MANILA','MANILA']})).reset_index()

In [15]:
provLookup = provLookup.drop(provLookup[provLookup['Province']=='SAMAR (WESTERN SAMAR)'].index.tolist())
provLookup = provLookup.drop(provLookup[provLookup['Province']=='WESTERN SAMAR'].index.tolist())
provLookup = provLookup.drop(provLookup[provLookup['Province']=='COTABATO (NORTH COTABATO)'].index.tolist())
provLookup = provLookup.drop(provLookup[provLookup['Province']=='NORTH COTABATO'].index.tolist())


In [16]:
psgc = psgc.merge(provLookup, on='provCode', how='left')

In [17]:
psgc = psgc.reset_index()

In [18]:
psgc = psgc[['code','location','provCode','Province']]
psgc = psgc.drop_duplicates()

In [19]:
psgc.loc[psgc['code']==99701000,'Province'] = 'Basilan'
psgc.loc[psgc['code']==129804000,'Province'] = 'Maguindanao'

#### Standardize column names

In [20]:
mun = mun.rename(columns={'City or municipality': 'location', 'Province Name': 'Province'})

In [21]:
mun['location'] = [str(item).upper() for item in mun['location']]
mun['Province'] = [str(item).title() for item in mun['Province']]

mun['location'] = [re.sub(' \((.*?)\)','',item) for item in mun['location']]
mun['location'] = [re.sub(' CITY$','',item) for item in mun['location']]


In [22]:
psgc['location'] = [str(item).upper() for item in psgc['location']]
psgc['Province'] = [str(item).title() for item in psgc['Province']]

psgc['location'] = [re.sub(' \((.*?)\)','',item) for item in psgc['location']]
psgc['location'] = [re.sub(' CITY$','',item) for item in psgc['location']]

In [23]:
changePSGC = [('ALFONSO CASTANEDA','ALFONSO CASTAÑEDA'),
('TONGKIL','BANGUINGUI'),
('BULACAN','BULAKAN'),
('PAGAGAWAN','DATU MONTAWAL'),
('GEN. MARIANO ALVAREZ','GENERAL MARIANO ALVAREZ'),
('GEN. S.K. PENDATUN','GENERAL SALIPADA K. PENDATUN'),
('IMPASUG-ONG','IMPASUGONG'),
 ('JALA-JALA','JALAJALA'),
 ('Leon T. Postigo','LEON B. POSTIGO'),
 ('MA-AYON','MAAYON'),
 ('PIGKAWAYAN','PIGCAWAYAN'),
 ('SAPI-AN','SAPIAN'),
 ('SEN. NINOY AQUINO','SENATOR NINOY AQUINO'),
 ('TALITAY','SULTAN SUMAGKA')]

In [24]:
psgc.loc[(psgc['location']=='SAN JOSE') & (psgc['Province']=='Antique'),'location'] = 'SAN JOSE DE BUENAVISTA'
psgc.loc[psgc['location']=='LEON T. POSTIGO','location'] = 'LEON B. POSTIGO'

In [25]:
for item in changePSGC:
    psgc.loc[psgc['location']==item[0],'location'] = item[1]


In [26]:
psgc = psgc[['code','location','provCode','Province']]
psgc = psgc.drop_duplicates()

In [27]:
mun = mun[['location','Population','Area','PD','Province']]
mun = mun.drop_duplicates()

In [28]:
mun.shape

(1635, 5)

In [29]:
psgc.shape

(1732, 4)

#### Merging `psgc` and `mun`   
### `mun` 
   * `location`: Name
   * `Population`: Number of People (2015)
   * `Area`: Land Area (km^2)
   * `PD`: Population Density (pop_2015/km^2)
   * `Barangay`: Number of Barangays
   * `interlevel`: Municipality, City
   * `Province`: Name of Province
   * `index`: ID no
   * `code`: PSGC Number

In [30]:
set(psgc['Province']).difference(set(mun['Province']))

{'Compostela Valley', 'Manila'}

In [31]:
set(mun['Province']).difference(set(psgc['Province']))

{'Davao De Oro',
 'Nan',
 'Ncr, 2Nd District',
 'Ncr, 3Rd District',
 'Ncr, 4Th District',
 'Ncr, City Of Manila, 1St District'}

In [32]:
changeProv = [
    ('Davao De Oro','Compostela Valley'),
    ('Ncr, 2Nd District','Manila'),
    ('Ncr, 3Rd District','Manila'),
    ('Ncr, 4Th District','Manila'),
    ('Ncr, City Of Manila, 1St District','Manila')
]

for item in changeProv:
    mun.loc[mun['Province']==item[0],'Province'] = item[1]


In [33]:
mun=mun.merge(psgc,
              on=['Province','location'],
              how='right')

In [34]:
mun = mun.drop(829)
mun = mun.reset_index()

In [35]:
# Check for missing population data
for i in mun[mun['Population'].isnull()].index.tolist():
    print(mun.loc[i,['location','Province']])

location        ESPIRITU
Province    Ilocos Norte
Name: 1634, dtype: object
location    CONCEPCION
Province    Ilocos Sur
Name: 1635, dtype: object
location       ANGKAKI
Province    Ilocos Sur
Name: 1636, dtype: object
location        BAUGEN
Province    Ilocos Sur
Name: 1637, dtype: object
location         LAPOG
Province    Ilocos Sur
Name: 1638, dtype: object
location      FAIRE
Province    Cagayan
Name: 1639, dtype: object
location    MAGSAYSAY
Province      Isabela
Name: 1640, dtype: object
location      BIGAA
Province    Bulacan
Name: 1641, dtype: object
location    BITULOK & SABANI
Province         Nueva Ecija
Name: 1642, dtype: object
location         PAPAYA
Province    Nueva Ecija
Name: 1643, dtype: object
location     SEXMOAN
Province    Pampanga
Name: 1644, dtype: object
location    MENDEZ-NUÑEZ
Province          Cavite
Name: 1645, dtype: object
location    AURORA
Province    Quezon
Name: 1646, dtype: object
location    MONTALBAN
Province        Rizal
Name: 1647, dtype: objec

### `subData` data
#### Merging `subData` and `mun` using `psgc`

In [60]:
subData = pd.read_csv('/Users/lorenzoflores/Desktop/Nerve/Hospitals/Hospitals.csv')

In [61]:
subData = subData[['Facility Name','Ownership Major Classification',
                   'Ownership Sub-Classification for Government facilities',
                   'Region Name','Province Name','City/Municipality Name',
                   'Hospital Licensing Status','Service Capability',
                  'Bed Capacity','lat','lon']]

subData = subData.rename(columns={'City/Municipality Name':'location','Province Name':'Province'})

#### Fix names of muni/city

In [62]:
subData['location'] = [item.replace(' (Capital)','').replace(' CITY','').replace('CITY OF ','').replace('?','Ñ') for item in subData['location']]
subData['location'] = [re.sub(' \((.*?)\)','',item) for item in subData['location']]
subData['Province'] = [item.title() for item in subData['Province']]
subData['Province'] = [re.sub(' \((.*?)\)','',item) for item in subData['Province']]


In [63]:
set(subData['location']).difference(set(mun['location']))

{'BULACAN',
 'GEN. MARIANO ALVAREZ',
 'ISLAND GARDEN OF SAMAL',
 'JALA-JALA',
 'TONDO I / II'}

In [64]:
changeSub = [
 ('BULACAN','BULAKAN'),
 ('GEN. MARIANO ALVAREZ','GENERAL MARIANO ALVAREZ'),
 ('ISLAND GARDEN OF SAMAL','SAMAL'),
 ('JALA-JALA','JALAJALA'),
 ('PIGKAWAYAN','PIGCAWAYAN'),
 ('SCIENCE OF MUÑOZ','MUÑOZ'),
 ('SEN. NINOY AQUINO','SENATOR NINOY AQUINO'),
 ('TONDO I / II','TONDO I/II'),
 ('TONGKIL','BANGUINGUI')]

for item in changeSub:
    subData.loc[subData['location']==item[0],'location'] = item[1]


In [65]:
subData.loc[(subData['Province']=='Antique') & (subData['location']=='SAN JOSE'),'location'] = 'SAN JOSE DE BUENAVISTA'


In [66]:
#set(mun['Province']).difference(set(subData['Province']))

In [67]:
#set(subData['Province']).difference(set(mun['Province']))

In [68]:
#set(subData['location']).difference(set(mun['location']))

In [69]:
subData.loc[subData['Province']=='City Of Isabela','Province'] = 'Basilan'
subData.loc[subData['Province']=='Cotabato City','Province'] = 'Maguindanao'
subData.loc[subData['Province']=='Ncr, Second District','Province'] = 'Manila'
subData.loc[subData['Province']=='Ncr, Third District','Province'] = 'Manila'
subData.loc[subData['Province']=='Ncr, Fourth District','Province'] = 'Manila'
subData.loc[subData['Province']=='Ncr, City Of Manila, First District','Province'] = 'Manila'
subData.loc[(subData['Province']=='Antique') & (subData['location']=='SAN JOSE'),'City/Municipality Name'] = 'SAN JOSE DE BUENAVISTA'

In [70]:
subData = pd.merge(subData, 
                   mun[['Province','location','code']], 
                   on=['location','Province'], 
                   how='left')

In [71]:
# Data Checking
subData['codenan'] = [1 if math.isnan(item) else 0 for item in subData['code']]
subData[subData['codenan']==1][['Province','location']]

,Province,location


In [72]:
subData['code'] = subData['code'].astype(int)

In [198]:
subData.to_csv("subData.csv")

## Bed and Facility Counts

In [87]:
subData[['code',
         'Facility Name']].groupby('code').count().to_csv('/Users/lorenzoflores/Desktop/Nerve/Hospitals/FacilityCount.csv')


In [88]:
subData[['code',
         'Bed Capacity']].groupby(['code']).sum().to_csv('/Users/lorenzoflores/Desktop/Nerve/Hospitals/BedCount.csv')


### Summary Statistics

In [138]:
bedTemp = pd.merge(mun, subData[['code',
         'Bed Capacity']].groupby(['code']).sum(), on='code', how='left')

In [139]:
bedTemp['Bed Capacity'] = bedTemp['Bed Capacity'].fillna(0)

In [140]:
bedTemp['ratio'] = 1000*(bedTemp['Bed Capacity']/bedTemp['Population'])

In [141]:
bedTemp = bedTemp[bedTemp['ratio'].notnull()]

In [142]:
#Overall
np.nansum(bedTemp['ratio'])/len(bedTemp['ratio'])

0.41254187303829865

In [199]:
bedTemp['Urban'] = [1 if item in ['Manila','Cebu','Cavite','Bulacan', 
                                  'Negros Occidental', 'Laguna', 'Pangasinan', 
                                  'Rizal', 'Batangas', 'Pampanga']
                   else 0 for item in bedTemp['Province']]

In [200]:
np.nansum(bedTemp[bedTemp['Province']=='Manila']['ratio'])/len(bedTemp[bedTemp['Province']=='Manila']['ratio'])


2.2116827926952047

In [201]:
np.nansum(bedTemp[bedTemp['Urban']==1]['ratio'])/len(bedTemp[bedTemp['Urban']==1]['ratio'])


0.5619026288040663

In [202]:
np.nansum(bedTemp[bedTemp['Urban']==0]['ratio'])/len(bedTemp[bedTemp['Urban']==0]['ratio'])


0.37936300657425

## Municipality Polygons

In [207]:
import json
with open('/Users/lorenzoflores/Desktop/Nerve/Hospitals/MuniCities.json') as json_file:
    municipality = json.load(json_file)

#### Clean polygon names same as `subData`, `psgc`, `mun`

In [208]:
lst = []
lst2 = []
for i in range(1647):    
    location = municipality['features'][i]['properties']['NAME_2'].upper()
    location = location.replace(' (Capital)','').replace(' CITY','').replace('CITY OF ','').replace('?','Ñ')
    location = re.sub(' \((.*?)\)','',location)
    
    municipality['features'][i]['properties']['NAME_2'] =  location

    Province = municipality['features'][i]['properties']['PROVINCE'].title()
    Province = re.sub(' \((.*?)\)','',Province) 
    
    municipality['features'][i]['properties']['PROVINCE'] = Province
    
    lst.append(location)
    lst2.append(Province)

In [209]:
tempPol = pd.DataFrame({'location':lst,
                        'Province':lst2})

#### Identify naming differences between `municipalities` and `mun`

In [210]:
polygonList = []
for i in range(tempPol.shape[0]):
    polygonList.append(tempPol['location'][i]+', '+tempPol['Province'][i])

In [211]:
munList = []
for i in range(mun.shape[0]):
    munList.append(mun['location'][i]+', '+mun['Province'][i])

In [212]:
set(polygonList).difference(set(munList))

{'ALAMADA, North Cotabato',
 'ALBUQUERQUE, Bohol',
 'ALEOSAN, North Cotabato',
 'ALFONSO CASTANEDA, Nueva Vizcaya',
 'ANTIPAS, North Cotabato',
 'ARAKAN, North Cotabato',
 'BACOLOD KALAWI, Lanao Del Sur',
 'BANISILAN, North Cotabato',
 'BARIRA, Shariff Kabunsuan',
 'BATO LAKE, Albay',
 'BATO LAKE, Camarines Sur',
 'BUHI LAKE, Camarines Sur',
 'BULACAN, Bulacan',
 'BULDON, Shariff Kabunsuan',
 'BULUAN LAKE, Maguindanao',
 'BULUAN LAKE, Sultan Kudarat',
 'BUTIG, Lanao Del Sur',
 'CARMEN, North Cotabato',
 'CORDOBA, Cebu',
 'DANAO LAKE, Cebu',
 'DAPAO LAKE, Lanao Del Sur',
 'DATU ABDULLAH SANKI, Maguindanao',
 'DATU BLAH T. SINSUAT, Shariff Kabunsuan',
 'DATU ODIN SINSUAT, Shariff Kabunsuan',
 'DON MARCELINO, Davao Del Sur',
 'DUENAS, Iloilo',
 'GEN. S. K. PENDATUN, Maguindanao',
 'HINUNANGAN, Quezon',
 'IMPASUG-ONG, Bukidnon',
 'JALA-JALA, Rizal',
 'JETAFE, Bohol',
 'JOSE ABAD SANTOS, Davao Del Sur',
 'KABACAN, North Cotabato',
 'KABUNTALAN, Shariff Kabunsuan',
 'KALIBATO LAKE, Laguna',


In [213]:
set(munList).difference(set(polygonList))

{'ALAMADA, Cotabato',
 'ALBOR, Dinagat Islands',
 'ALBURQUERQUE, Bohol',
 'ALEOSAN, Cotabato',
 'ALFONSO CASTAÑEDA, Nueva Vizcaya',
 'AMAI MANABILANG, Lanao Del Sur',
 'ANAO-AON, Surigao Del Norte',
 'ANGKAKI, Ilocos Sur',
 'ANTIPAS, Cotabato',
 'ARAKAN, Cotabato',
 'ASIA, Negros Occidental',
 'AURORA, Quezon',
 'AYUQUITAN, Negros Oriental',
 'BACOLOD GRANDE, Lanao Del Sur',
 'BACOLOD-KALAWI, Lanao Del Sur',
 'BACUIT, Palawan',
 'BALIMBING, Tawi-Tawi',
 'BANGUINGUI, Sulu',
 'BANISILAN, Cotabato',
 'BARIRA, Maguindanao',
 'BAUGEN, Ilocos Sur',
 'BAYAG, Apayao',
 'BIGAA, Bulacan',
 'BINONDO, Manila',
 'BITULOK & SABANI, Nueva Ecija',
 'BORJA, Bohol',
 'BUGHO, Leyte',
 'BULAKAN, Bulacan',
 'BULDON, Maguindanao',
 'CABALIAN, Southern Leyte',
 'CAGAYAN DE TAWI-TAWI, Tawi-Tawi',
 'CALOLBON, Catanduanes',
 'CALOOCAN, Manila',
 'CARMEN, Cotabato',
 'CONCEPCION, Ilocos Sur',
 'CORDOVA, Cebu',
 'DADIANGAS, South Cotabato',
 'DAO, Antique',
 'DATU ABDULLAH SANGKI, Maguindanao',
 'DATU BLAH T. SIN

In [214]:
changeProvList = [('North Cotabato','Cotabato'),
('Metropolitan Manila','Manila'),
('Shariff Kabunsuan','Maguindanao')]


In [215]:
changeList = [('ALBURQUERQUE','ALBUQUERQUE'),
('ALFONSO CASTAÑEDA','ALFONSO CASTANEDA'),
('BACOLOD-KALAWI','BACOLOD KALAWI'),
('BULAKAN','BULACAN'),
('CALOOCAN','KALOOKAN'),
('CORDOVA','CORDOBA'),
('DATU ABDULLAH SANGKI','DATU ABDULLAH SANKI'),
('DUEÑAS','DUENAS'),
('GENERAL SALIPADA K. PENDATUN','GEN. S. K. PENDATUN'),
('GETAFE','JETAFE'),
('JALAJALA','JALA-JALA'),
('IMPASUGONG','IMPASUG-ONG'),
('LUMBACA-UNAYAN','LUMBACA UNAYAN'),
('MAAYON','MA-AYON'),
('MATAASNAKAHOY','MATAAS NA KAHOY'),
('OZAMIZ','OZAMIS'),
('PIGCAWAYAN','PIGKAWAYAN'),
('PINAMUNGAJAN','PINAMUNGAHAN'),
('PIÑAN','PINAN'),
('POZORRUBIO','POZZORUBIO'),
('PRESIDENT CARLOS P. GARCIA','PRES. CARLOS P. GARCIA'),
('PRESIDENT MANUEL A. ROXAS','PRES. MANUEL A. ROXAS'),
('SAGÑAY','SAGNAY'),
('SAPIAN','SAPI-AN'),
('SENATOR NINOY AQUINO','SEN. NINOY AQUINO'),
('SERGIO OSMEÑA SR.','SERGIO OSMENA SR.'),
('SOFRONIO ESPAÑOLA','SOFRONIO ESPANOLA'),
('BANGUINGUI','TONGKIL'),
('SANTO NIÑO','SANTO NINO'),
('LEON B. POSTIGO','BACUNGAN'),
('AMAI MANABILANG','BUMBARAN'),
('SULTAN SUMAGKA','TALITAY')]

In [216]:
changeName = [('DON MARCELINO, Davao Del Sur','Davao Occidental'),
              ('JOSE ABAD SANTOS, Davao Del Sur','Davao Occidental'),
              ('MALITA, Davao Del Sur','Davao Occidental'),
              ('SANTA MARIA, Davao Del Sur','Davao Occidental'),
              ('SARANGANI, Davao Del Sur','Davao Occidental')]

In [217]:
for i in range(1647):
    location = municipality['features'][i]['properties']['NAME_2']
    Province = municipality['features'][i]['properties']['PROVINCE']
    name = location+', '+Province
    
    for item in changeList:
        if item[1]==location:
            municipality['features'][i]['properties']['NAME_2'] = item[0]
    for item in changeProvList:
        if item[0]==Province:
            municipality['features'][i]['properties']['PROVINCE'] = item[1]
    for item in changeName:
        if name==item[0]:
            municipality['features'][i]['properties']['PROVINCE'] = item[1]
    if name == 'SAN JOSE, Antique':
        municipality['features'][i]['properties']['NAME_2'] = 'SAN JOSE DE BUENAVISTA'

#### Recheck discrepancies

In [218]:
lst = []
lst2 = []
for i in range(1647):    
    location = municipality['features'][i]['properties']['NAME_2']
    Province = municipality['features'][i]['properties']['PROVINCE']
    lst.append(location)
    lst2.append(Province)

In [219]:
tempPol = pd.DataFrame({'location':lst,
                        'Province':lst2})

In [220]:
polygonList = []
for i in range(tempPol.shape[0]):
    polygonList.append(tempPol['location'][i]+', '+tempPol['Province'][i])

In [221]:
set(polygonList).difference(set(munList))

{'BATO LAKE, Albay',
 'BATO LAKE, Camarines Sur',
 'BUHI LAKE, Camarines Sur',
 'BULUAN LAKE, Maguindanao',
 'BULUAN LAKE, Sultan Kudarat',
 'BUTIG, Lanao Del Sur',
 'DANAO LAKE, Cebu',
 'DAPAO LAKE, Lanao Del Sur',
 'HINUNANGAN, Quezon',
 'KALIBATO LAKE, Laguna',
 'LAGUNA LAKE, Laguna',
 'LAKEWOOD LAKE, Zamboanga Del Sur',
 'LANAO LAKE, Lanao Del Sur',
 'MAINIT LAKE, Agusan Del Norte',
 'MAINIT LAKE, Surigao Del Norte',
 'NAUJAN LAKE, Oriental Mindoro',
 'PAGAGAWAN, Maguindanao',
 'PALAKPAKIN LAKE, Laguna',
 'PAOAY LAKE, Ilocos Norte',
 'SAMPALOC LAKE, Laguna',
 'TAAL LAKE, Batangas',
 'WATERBODY, Laguna'}

In [222]:
set(munList).difference(set(polygonList))

{'ALBOR, Dinagat Islands',
 'ANAO-AON, Surigao Del Norte',
 'ANGKAKI, Ilocos Sur',
 'ASIA, Negros Occidental',
 'AURORA, Quezon',
 'AYUQUITAN, Negros Oriental',
 'BACOLOD GRANDE, Lanao Del Sur',
 'BACUIT, Palawan',
 'BACUNGAN, Zamboanga Del Norte',
 'BALIMBING, Tawi-Tawi',
 'BAUGEN, Ilocos Sur',
 'BAYAG, Apayao',
 'BIGAA, Bulacan',
 'BINONDO, Manila',
 'BITULOK & SABANI, Nueva Ecija',
 'BORJA, Bohol',
 'BUGHO, Leyte',
 'BUMBARAN, Lanao Del Sur',
 'CABALIAN, Southern Leyte',
 'CAGAYAN DE TAWI-TAWI, Tawi-Tawi',
 'CALOLBON, Catanduanes',
 'CONCEPCION, Ilocos Sur',
 'DADIANGAS, South Cotabato',
 'DAO, Antique',
 'DATU HOFFER AMPATUAN, Maguindanao',
 'DATU MONTAWAL, Maguindanao',
 'DATU SALIBO, Maguindanao',
 'DINAIG, Maguindanao',
 'DON MARIANO MARCOS, Misamis Occidental',
 'DON MARIANO MARCOS, Zamboanga Del Sur',
 'DON VICTORIANO CHIONGBIAN , Misamis Occidental',
 'DOÑA ALICIA, Compostela Valley',
 'ERMITA, Manila',
 'ESPIRITU, Ilocos Norte',
 'FAIRE, Cagayan',
 'GATA, Lanao Del Sur',
 'H

#### Fill in missing population density in `mun` 

In [223]:
mun.loc[mun['location']=='HADJI MUHTAMAD','Area'] = 173.22
mun.loc[mun['location']=='LAMITAN','Area'] = 354.45
mun.loc[mun['location']=='SUMISIP','Area'] = 567.60
mun.loc[mun['location']=='TUBURAN','Area'] = 546.00
mun.loc[mun['location']=='SIBUTU','Area'] = 285.32

mun.loc[mun['location']=='HADJI MUHTAMAD','PD'] = 25085/173.22
mun.loc[mun['location']=='LAMITAN','PD'] = 74782/354.45
mun.loc[mun['location']=='SUMISIP','PD'] = 41730/567.60
mun.loc[mun['location']=='TUBURAN','PD'] = 20207/546.00
mun.loc[mun['location']=='SIBUTU','PD'] = 30387/285.32




#### Add psgc, population, and population density to `municipality`

In [224]:
count = 0
for i in range(1647):
    prov = municipality['features'][i]['properties']['PROVINCE']
    name = municipality['features'][i]['properties']['NAME_2']

    try:
        temp = mun[(mun['location']==name) & (mun['Province']==prov)].reset_index() 
        municipality['features'][i]['properties']['PD'] = int(temp['PD'][0])
        municipality['features'][i]['properties']['population'] = int(temp['Population'][0])
        municipality['features'][i]['properties']['psgc'] = int(temp['code'][0])
        continue
    except:
        pass
    
    try:
        print("Province mismatch on "+str(i)+": "+name+', '+prov)
        temp = mun[mun['location']==name].reset_index() 
        municipality['features'][i]['properties']['PD'] = int(temp['PD'][0])
        municipality['features'][i]['properties']['population'] = int(temp['Population'][0])
        municipality['features'][i]['properties']['psgc'] = int(temp['code'][0])
    except:
        print("Complete error on: "+str(i)+": "+name+', '+prov)
        municipality['features'][i]['properties']['PD'] = 'NA'
        municipality['features'][i]['properties']['population'] = 'NA'
        municipality['features'][i]['properties']['psgc'] = 'NA'
        count += 1


Province mismatch on 16: MAINIT LAKE, Agusan Del Norte
Complete error on: 16: MAINIT LAKE, Agusan Del Norte
Province mismatch on 157: BATO LAKE, Albay
Complete error on: 157: BATO LAKE, Albay
Province mismatch on 265: TAAL LAKE, Batangas
Complete error on: 265: TAAL LAKE, Batangas
Province mismatch on 431: BATO LAKE, Camarines Sur
Complete error on: 431: BATO LAKE, Camarines Sur
Province mismatch on 434: BUHI LAKE, Camarines Sur
Complete error on: 434: BUHI LAKE, Camarines Sur
Province mismatch on 525: DANAO LAKE, Cebu
Complete error on: 525: DANAO LAKE, Cebu
Province mismatch on 666: PAOAY LAKE, Ilocos Norte
Complete error on: 666: PAOAY LAKE, Ilocos Norte
Province mismatch on 805: KALIBATO LAKE, Laguna
Complete error on: 805: KALIBATO LAKE, Laguna
Province mismatch on 806: LAGUNA LAKE, Laguna
Complete error on: 806: LAGUNA LAKE, Laguna
Province mismatch on 818: PALAKPAKIN LAKE, Laguna
Complete error on: 818: PALAKPAKIN LAKE, Laguna
Province mismatch on 822: SAMPALOC LAKE, Laguna
Comp

In [239]:
with open('/Users/lorenzoflores/Desktop/Nerve/Hospitals/munPop.json', 'w') as outfile:
    json.dump(municipality, outfile)

## Nearest Hospitals

#### Geotagging Municipalities

In [234]:
# mun['lat'] = np.repeat(None, mun.shape[0])
# mun['lon'] = np.repeat(None, mun.shape[0])

# errNo = []
# for i in range(mun.shape[0]):
#     if i%100 == 0:
#         print("Now on: "+str((i/mun.shape[0])*100)+"%")
    
#     add = mun['location'][i] + ', ' + mun['Province'][i] + ', ' + 'Philippines'
#     temp = geoTag(add)
#     if temp != 0:
#         mun.loc[i,'lat'] = temp[0]
#         mun.loc[i,'lon'] = temp[1]
    
#     else:
#         print('Error on '+str(i)+': '+add)
#         errNo.append(i)       
                  
# mun.to_csv("munWithCoords.csv")


Now on: 0.0%
Now on: 5.7770075101097635%
Now on: 11.554015020219527%
Now on: 17.33102253032929%
Now on: 23.108030040439054%
Now on: 28.885037550548816%
Now on: 34.66204506065858%
Now on: 40.43905257076834%
Now on: 46.21606008087811%
Now on: 51.99306759098788%
Now on: 57.77007510109763%
Now on: 63.547082611207394%
Now on: 69.32409012131716%
Now on: 75.10109763142691%
Now on: 80.87810514153668%
Now on: 86.65511265164645%
Now on: 92.43212016175622%
Now on: 98.20912767186597%


In [214]:
mun = pd.read_csv("munWithCoords.csv")

In [215]:
index = []
mins = []
km1 = []
km5 = []
km10 = []
for i in range(mun.shape[0]):
    if i%100==0:
        print(i)
    temp = np.power(subData['lat']-mun['lat'][i],2)+np.power(subData['lon']-mun['lon'][i],2)
    temp = [math.sqrt(item)/0.009009 for item in temp]
    index.append(np.argmin(temp))
    mins.append(np.amin(temp))
    km1.append(sum(k <= 1 for k in temp))
    km5.append(sum(k <= 5 for k in temp))
    km10.append(sum(k <= 10 for k in temp))


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700


In [216]:
mun['minIndex'] = index
mun['closestDistance'] = mins



In [217]:
munTemp = mun
#Overall
np.nansum(munTemp['closestDistance'])/len(munTemp['closestDistance'])


12.562757057286102

In [218]:
munTemp['Urban'] = [1 if item in ['Manila','Cebu','Cavite','Bulacan', 
                                  'Negros Occidental', 'Laguna', 'Pangasinan', 
                                  'Rizal', 'Batangas', 'Pampanga']
                   else 0 for item in munTemp['Province']]


In [219]:
np.nansum(munTemp[munTemp['Province']=='Manila']['closestDistance'])/len(munTemp[munTemp['Province']=='Manila']['closestDistance'])


0.6688863294033827

In [220]:
np.nansum(munTemp[munTemp['Urban']==1]['closestDistance'])/len(munTemp[munTemp['Urban']==1]['closestDistance'])


6.363483331400665

In [221]:
np.nansum(munTemp[munTemp['Urban']==0]['closestDistance'])/len(munTemp[munTemp['Urban']==0]['closestDistance'])


13.963301192241808

In [232]:
munTemp[munTemp['closestDistance']>200]

,Unnamed: 0,index,location,Population,Area,PD,Province,code,provCode,lat,lon,minIndex,closestDistance,Urban
1540,1540,1541,MAPUN,26597.0,181.29,147.0,Tawi-Tawi,157003000,157000000,6.997029,118.474717,697,205.994844,0
1548,1548,1549,TURTLE ISLANDS,4727.0,62.5,76.0,Tawi-Tawi,157008000,157000000,6.217003,118.252662,21,203.188229,0
1720,1720,1721,CAGAYAN DE TAWI-TAWI,NaN,NaN,NaN,Tawi-Tawi,157003000,157000000,6.997029,118.474717,697,205.994844,0


In [192]:
mun.to_csv("/Users/lorenzoflores/Desktop/Nerve/Hospitals/munDist.csv")

## Covid Centers

In [233]:
path = '/Users/lorenzoflores/Desktop/Nerve/Hospitals/CovidList'
extension = 'csv'
os.chdir(path)
titles = glob.glob('*.{}'.format(extension))


In [235]:
cov = pd.read_csv(titles[0])
for item in titles[1:]:
    cov = cov.append(pd.read_csv(item))

In [244]:
cov = cov[cov['COVID-19sorted descending']=='✔']

In [249]:
set(cov['Name']).difference(subData['Facility Name'])

{"DE VERA'S MEDICAL CENTER, INC.",
 'DR. RONALD P. GUZMAN MEDICAL CENTER',
 'TANCHULING HOSPITAL'}

In [250]:
for item in subData['Facility Name']:
    if 'vera' in item.lower():
        print(item)
    if 'tanchuling' in item.lower():
        print(item)
    if 'ronald' in item.lower():
        print(item)

DE VERA MEDICAL CENTER, INC.
DR. RONALD P. GUZMAN MEDICAL CENTER, INC.
TALAVERA GENERAL HOSPITAL
TANCHULING GENERAL HOSPITAL
VILLANUEVA-TANCHULING MATERNITY AND GENERAL HOSPITAL INCORPORATED
RIVERA MEDICAL CENTER, INC.


In [254]:
subData = pd.merge(subData,cov[['Name','COVID-19sorted descending']],
                   left_on='Facility Name',
                   right_on='Name',
                   how='left')

In [256]:
subData['COVID-19sorted descending'] = [1 if item=='✔' else 0 for item in subData['COVID-19sorted descending']]


In [263]:
covList = subData.index[subData['COVID-19sorted descending']==1].tolist()
for i in covList:
    subData.loc[i,'Service Capability'] = subData.loc[i,'Service Capability'] + ', Covid Hosp.'

In [264]:
subData['Service Capability']

0                    Level 1
1                    Level 1
2                    Level 1
3                    Level 1
4                    Level 1
                ...         
1593                       0
1594                 Level 1
1595    Level 2, Covid Hosp.
1596    Level 2, Covid Hosp.
1597    Level 2, Covid Hosp.
Name: Service Capability, Length: 1598, dtype: object

In [269]:
subData.loc[subData['Facility Name']=='DR. RONALD P. GUZMAN MEDICAL CENTER, INC.',:]

,Facility Name,Ownership Major Classification,Ownership Sub-Classification for Government facilities,Region Name,Province,location,Hospital Licensing Status,Service Capability,Bed Capacity,lat,lon,City/Municipality Name,code,codenan,Name,COVID-19sorted descending
223,"DR. RONALD P. GUZMAN MEDICAL CENTER, INC.",Private,NaN,REGION II (CAGAYAN VALLEY),Cagayan,TUGUEGARAO,With License,"Level 2, Covid Hosp.",100.0,17.649541,121.755894,NaN,21529000,0,"DR. RONALD P. GUZMAN MEDICAL CENTER, INC.",1


In [270]:
subData.to_csv('subData.csv')

## Filtered Data Set for ArcGIS

In [4]:
subArc = pd.read_csv('subData.csv')

In [8]:
subArc = subArc[(subArc['Hospital Licensing Status']=='With License') & (subArc['Service Capability'].isin(['Level 1','Level 2','Level 3']))]

In [16]:
subArc[subArc['Province']=='Manila'].groupby('location').count()

,Unnamed: 0,Facility Name,Ownership Major Classification,Ownership Sub-Classification for Government facilities,Region Name,Province,Hospital Licensing Status,Service Capability,Bed Capacity,lat,lon,City/Municipality Name,code,codenan
location,,,,,,,,,,,,,,
BINONDO,1,1,1,1,1,1,1,1,1,1,1,0,1,1
CALOOCAN,9,9,9,2,9,9,9,9,9,9,9,0,9,9
ERMITA,3,3,3,1,3,3,3,3,3,3,3,0,3,3
INTRAMUROS,1,1,1,0,1,1,1,1,1,1,1,0,1,1
LAS PIÑAS,10,10,10,1,10,10,10,10,10,10,10,0,10,10
MAKATI,4,4,4,1,4,4,4,4,4,4,4,0,4,4
MALABON,2,2,2,1,2,2,2,2,2,2,2,0,2,2
MALATE,1,1,1,1,1,1,1,1,1,1,1,0,1,1
MANDALUYONG,4,4,4,2,4,4,4,4,4,4,4,0,4,4


In [17]:
subArc.to_csv('subArc.csv')

### Poverty Statistics
* column 3: `Incidence09`
* column 4: `Incidence12`
* column 5: `Incidence15`

In [75]:
pov = pd.read_csv("/Users/lorenzoflores/Desktop/Nerve/Hospitals/povertyIndex.csv")

In [76]:
pov.columns = pov.loc[0,:]

In [77]:
pov = pov.iloc[pov['PSGC'].dropna(axis=0).index.to_list(),[0,2,5]].drop(0,axis=0).reset_index()

In [78]:
pov['PSGC'] = [int(item+'000') for item in pov['PSGC']]

In [79]:
munPoverty = pd.read_csv("/Users/lorenzoflores/Desktop/Nerve/Hospitals/munWithCoords.csv")

In [83]:
munPoverty['provCode'] = [int(item) for item in munPoverty['provCode']]

In [84]:
pov = pov.rename(columns={pov.columns[3]:'Poverty'})

In [95]:
pov = pd.merge(munPoverty[['location','code']], pov[['PSGC','Municipality/City','Poverty']], left_on='code',
         right_on='PSGC', how='outer')

In [101]:
# Poverty incidence in Manila
(sum(pov.loc[pov['code'].index[pov['code'].apply(np.isnan)],:]['Poverty'])-65.1)/14

5.714285714285714

In [112]:
pov = pov.drop(pov['code'].index[pov['code'].apply(np.isnan)], axis=0)

In [116]:
pov['code'] = [str(int(item)) for item in pov['code']]

In [117]:
pov['code']

0       140101000
1       140102000
2       140103000
3       140104000
4       140105000
          ...    
1898     98312000
1899     98313000
1900     98314000
1901     98315000
1902     98316000
Name: code, Length: 1903, dtype: object

In [118]:
pov.to_csv('/Users/lorenzoflores/Desktop/Nerve/Hospitals/poverty.csv')